**DRİVE BAĞLANMA**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**KÜTÜPHANELERİ YÜKLEME**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import keras
from tqdm import tqdm
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from keras.models import Model,Sequential, load_model
from tensorflow.keras import Input
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, AveragePooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.applications.vgg16 import VGG16


**VERİ ÖN YÜKLEME VE İŞLEME**

In [ ]:
class_type = ['COVID','non-COVID']
data_dir = '/content/drive/MyDrive/deep learning/data'
train_dir = os.path.join(data_dir)

train_data = []
for defects_id, sp in enumerate(class_type):
    print(defects_id, sp)
    for file in os.listdir(os.path.join(train_dir, sp)):
      train_data.append(['{}/{}'.format(sp,file),defects_id, sp])

print(train_data)
train = pd. DataFrame(train_data, columns = ['File', 'DiseaseID', 'Disease Type'])
print(train)
SEED = 42
train = train.sample(frac=1, random_state = SEED)
train_index = np.arange(len(train))
print(train)
print(train_index)

0 COVID
1 non-COVID
[['COVID/Covid (1).png', 0, 'COVID'], ['COVID/Covid (100).png', 0, 'COVID'], ['COVID/Covid (10).png', 0, 'COVID'], ['COVID/Covid (12).png', 0, 'COVID'], ['COVID/Covid (11).png', 0, 'COVID'], ['COVID/Covid (14).png', 0, 'COVID'], ['COVID/Covid (2).png', 0, 'COVID'], ['COVID/Covid (16).png', 0, 'COVID'], ['COVID/Covid (17).png', 0, 'COVID'], ['COVID/Covid (23).png', 0, 'COVID'], ['COVID/Covid (24).png', 0, 'COVID'], ['COVID/Covid (25).png', 0, 'COVID'], ['COVID/Covid (22).png', 0, 'COVID'], ['COVID/Covid (18).png', 0, 'COVID'], ['COVID/Covid (19).png', 0, 'COVID'], ['COVID/Covid (26).png', 0, 'COVID'], ['COVID/Covid (27).png', 0, 'COVID'], ['COVID/Covid (13).png', 0, 'COVID'], ['COVID/Covid (20).png', 0, 'COVID'], ['COVID/Covid (15).png', 0, 'COVID'], ['COVID/Covid (21).png', 0, 'COVID'], ['COVID/Covid (28).png', 0, 'COVID'], ['COVID/Covid (44).png', 0, 'COVID'], ['COVID/Covid (3).png', 0, 'COVID'], ['COVID/Covid (38).png', 0, 'COVID'], ['COVID/Covid (37).png', 0, 'CO

Mounted at /content/drive


In [ ]:
IMAGE_SIZE = 128
def read_image(filepath):
    return cv2.imread(os.path.join(data_dir, filepath)) # Loading a color image is the default flag
# Resize image to target size
def resize_image(image, image_size):
    return cv2.resize(image.copy(), image_size, interpolation=cv2.INTER_AREA)

In [ ]:
X_train = np.zeros((train.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3))
for i, file in tqdm(enumerate(train['File'].values)):
    image = read_image(file)
    if image is not None:
        X_train[i] = resize_image(image, (IMAGE_SIZE, IMAGE_SIZE))
# Normalize the data
X_Train = X_train / 255.
print('Train Shape: {}'.format(X_Train.shape))

200it [01:24,  2.38it/s]

Train Shape: (200, 128, 128, 3)


In [ ]:
from keras.utils import to_categorical

Y_train = train['DiseaseID'].values
Y_train = to_categorical(Y_train)

**VERİ SETİNİ AYIRMA**

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X_Train, Y_train, test_size=0.2, random_state=SEED)

**PARAMETRELER**

In [ ]:
EPOCHS = 30
BATCH_SIZE=4
SIZE=128
N_ch=3

**VGG16 MODEL**

In [ ]:

from keras.applications.vgg16 import VGG16


def build_vgg():
    vgg = VGG16(weights='imagenet', include_top=False)

    input = Input(shape=(SIZE, SIZE, N_ch))
    x = Conv2D(3, (3, 3), padding='same')(input)
    x = vgg(x)
    x = Flatten(name="flatten")(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    # multi output
    output = Dense(2,activation = 'softmax', name='root')(x)
    # model
    model = Model(input,output)

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()

    return model

model_vgg = build_vgg()
model_vgg.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 3)       84        
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dropout (Dropout)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 batch_normalization (Batch  (None, 256)               1024  

**INCEPTIONRESNETV2 MODEL**

In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
def build_inres():
    inres = InceptionResNetV2(weights='imagenet', include_top=False)

    input = Input(shape=(SIZE, SIZE, N_ch))
    x = Conv2D(3, (3, 3), padding='same')(input)
    x = inres(x)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    # multi output
    output = Dense(2,activation = 'softmax', name='root')(x)
    model = Model(input,output)

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()

    return model


**DENSENET201 MODEL**

In [ ]:
from keras.applications.densenet import DenseNet121
def build_densenet():
    densenet = DenseNet121(weights='imagenet', include_top=False)

    input = Input(shape=(SIZE, SIZE, N_ch))
    x = Conv2D(3, (3, 3), padding='same')(input)
    x = densenet(x)
    x = GlobalAveragePooling2D()(x) # flatten()
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    # multi output
    output = Dense(2,activation = 'softmax', name='root')(x)

    # model
    model = Model(input,output)

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()

    return model

**EĞİTİM AŞAMASI**

In [ ]:
from keras.applications.vgg16 import VGG16

model_vgg = build_vgg()
#model_incep = build_inres()
model_dense = build_densenet()

#Fits the model on batches with real-time data augmentation
model_vgg.fit(X_train, Y_train, batch_size=BATCH_SIZE,
              epochs=EPOCHS,
             verbose=2,
            validation_data=(X_val, Y_val))

##model_incep.fit(X_train, Y_train, batch_size=BATCH_SIZE,
#               epochs=EPOCHS,
#               verbose=2,
#               validation_data=(X_val, Y_val))

model_dense.fit(X_train, Y_train, batch_size=BATCH_SIZE,
               epochs=EPOCHS,
               verbose=2,
               validation_data=(X_val, Y_val))

58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 3)       84        
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dropout (Dropout)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                              

**TEST İŞLEMİ**

In [ ]:
score_vgg = model_vgg.evaluate(X_val, Y_val, verbose=0)
#score_incep = model_incep.evaluate(X_val, Y_val, verbose=0)
score_dense = model_dense.evaluate(X_val, Y_val, verbose=0)

#print("Test loss:", score[0])
print("VGG Modelinin Doğruluk Skoru :", score_dense[1])
#print("InceptionResNetv2 Modelinin Doğruluk Skoru :", score_incep[1])
print("DenseNet201 Modelinin Doğruluk Skoru :", score_dense[1])

VGG Modelinin Doğruluk Skoru : 1.0
DenseNet201 Modelinin Doğruluk Skoru : 1.0


In [ ]:
y_pred_vgg = model_vgg.predict(X_val)
y_pred_incep = model_incep.predict(X_val)
y_pred_dense = model_dense.predict(X_val)

y_pred_vgg=np.argmax(y_pred_vgg, axis=1)
y_pred_incep=np.argmax(y_pred_incep, axis=1)
y_pred_dense=np.argmax(y_pred_dense, axis=1)

Y_val=np.argmax(Y_val, axis=1)

print("VGG16 MODEL SONUÇLARI")
print("Doğruluk: ",accuracy_score(Y_val, y_pred_vgg))
print("F1_Skor: ", f1_score(Y_val, y_pred_vgg))
print("Hassasiyet: ",precision_score(Y_val, y_pred_vgg))
print("Duyarlılık: ", recall_score(Y_val, y_pred_vgg))

print("InceptionResNetv2 MODEL SONUÇLARI")
print("Doğruluk: ",accuracy_score(Y_val, y_pred_incep))
print("F1_Skor: ", f1_score(Y_val, y_pred_incep))
print("Hassasiyet: ",precision_score(Y_val, y_pred_incep))
print("Duyarlılık: ", recall_score(Y_val, y_pred_incep))

print("DenseNet201 MODEL SONUÇLARI")
print("Doğruluk: ",accuracy_score(Y_val, y_pred_dense))
print("F1_Skor: ", f1_score(Y_val, y_pred_dense))
print("Hassasiyet: ",precision_score(Y_val, y_pred_dense))
print("Duyarlılık: ", recall_score(Y_val, y_pred_dense))

VGG16 MODEL SONUÇLARI
Doğruluk:  0.525
F1_Skor:  0.0
Hassasiyet:  0.0
Duyarlılık:  0.0
InceptionResNetv2 MODEL SONUÇLARI
Doğruluk:  0.9083333333333333
F1_Skor:  0.905982905982906
Hassasiyet:  0.8833333333333333
Duyarlılık:  0.9298245614035088
DenseNet201 MODEL SONUÇLARI
Doğruluk:  0.8666666666666667
F1_Skor:  0.8709677419354839
Hassasiyet:  0.8059701492537313
Duyarlılık:  0.9473684210526315


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# VGG16 accuracy plot (hist_vgg16 yerine diğerleri (hist_dense gibi) yazılabilir)
plt.plot(hist_vgg16.history['accuracy'])
plt.plot(hist_vgg16.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# VGG16 loss plot
plt.plot(hist_vgg16.history['loss'])
plt.plot(hist_vgg16.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()